In [ ]:
import dask
from dask.distributed import Client

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.image_analysis.delayed as delayed

In [ ]:
client = Client()
client

In [ ]:
def foo(x, y, z):
    out = f"x:{x} y:{y} z:{z}"
    print(f"OUT: {out}")
    return out

# Evicting too soon should error

In [ ]:
queue = delayed.DelayedQueue()
s = delayed.DelayedStore(queue)
s["a"] = "foo"
s["b"] = "bar"
s["c"] = "baz"
del s["a"]
# d = queue.delayed(foo, a, b, c)
d = s.setdefault("d", queue.delayed(foo, s["a"], s["b"], s["c"]))
d.result()

# Correct eviction

In [ ]:
queue = delayed.DelayedQueue()
s = delayed.DelayedStore(queue)
s["a"] = "foo"
s["b"] = "bar"
s["c"] = "baz"
# d = queue.delayed(foo, a, b, c)
d = s.setdefault("d", queue.delayed(foo, s["a"], s["b"], s["c"]))
del s["a"]
d.result()

## Does not evict in-use keys

In [ ]:
queue = delayed.DelayedQueue()
s = delayed.DelayedStore(queue)
a = s["a"]
b = s["b"]
c = s["c"]
# d = queue.delayed(foo, a, b, c)
d = s.setdefault("d", queue.delayed(foo, a, b, c))

In [ ]:
s["a"] = 1
s["b"] = 2
s["c"] = 3

In [ ]:
del s["a"]

In [ ]:
d.result()

# Evicts unused keys

In [ ]:
queue = delayed.DelayedQueue()
s = delayed.DelayedStore(queue)
a = s["a"]
b = s["b"]
c = s["c"]
# d = queue.delayed(foo, a, b, c)
d = s.setdefault("d", queue.delayed(foo, a, b, c))